<a href="https://colab.research.google.com/github/kmk4444/System_engineering/blob/main/Ragas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install langchain_openai ragas

None


In [ ]:
import os
from langchain_openai import ChatOpenAI
from ragas import evaluate, EvaluationDataset
from ragas.metrics import AnswerRelevancy, Faithfulness, FactualCorrectness, AnswerCorrectness
from ragas.llms import LangchainLLMWrapper

# OpenAI API Anahtarını Ayarla
os.environ["OPENAI_API_KEY"] = ""

# LLM Modelini Başlat
llm = ChatOpenAI(model="gpt-4o")  # GPT-4o değerlendirme için kullanılacak
evaluator_llm = LangchainLLMWrapper(llm)

# Sorular
questions = [
    "Sistem mühendisliği nedir ve temel amacı nedir?",
    "V-Model (V-Şeması) nedir ve hangi aşamaları içerir?",
    "Sistem gereksinimleri ile kullanıcı gereksinimleri arasındaki fark nedir?",
    "İhtiyaç analizi neden önemlidir?",
    "Trade-off analizi nedir ve neden yapılır?"
]

# ChatGPT-4o'nun Referans Cevapları
reference_answers = [
    "Sistem mühendisliği, karmaşık sistemlerin tasarım, geliştirme, entegrasyon ve yaşam döngüsü yönetimiyle ilgilenen disiplinler arası bir alanıdır. Temel amacı, sistemleri en verimli şekilde tasarlamak, sürekliliğini sağlamak ve uçtan uca yönetmektir.",
    "V-Model, sistem veya yazılım geliştirme yaşam döngüsünü tanımlamak için kullanılan bir modeldir. Sol kolunda gereksinim analizi ve tasarım faaliyetleri, sağ kolunda ise test ve doğrulama faaliyetleri bulunur. Ortada entegrasyon aşaması yer alır ve süreç, gereksinimlerden doğrulamaya kadar sistematik bir biçimde ilerler.",
    "Sistem gereksinimleri, bir sistemin teknik ve işlevsel niteliklerini tanımlar ve tasarım ekibi tarafından anlaşılması gereken ayrıntıları içerir. Kullanıcı gereksinimleri ise sistemin son kullanıcı tarafından nasıl kullanılacağını, hangi amaçlara hizmet etmesi gerektiğini ve kullanıcı deneyimi ile ilgili beklentileri belirtir.",
    "İhtiyaç analizi, sistemin doğru şekilde tasarlanabilmesi için tüm paydaşların taleplerini ve beklentilerini detaylı biçimde anlamayı amaçlar. Yanlış veya eksik gereksinimler projede gecikmelere, maliyet artışlarına ve hatta sistemin istenilen performansa ulaşamamasına neden olabilir.",
    "Trade-off analizi, bir sistemin tasarımı sırasında birbiriyle çelişen gereksinim veya kısıtlar arasında optimal dengeyi bulmaya çalışmaktır. Örneğin performans ile maliyet, hız ile enerji tüketimi, kalite ile süre gibi faktörler arasında seçim yapılması gerektiğinde trade-off analizi kullanılır."
]

# DeepSeek-r1'in Verdiği Cevaplar
deepseek_answers = [
    "Sistem mühendisliği, kullanıcıların ihtiyaçlarını karşılayacak şekilde tasarlanmış bir sistemin oluşturulmasını sağlayan disiplindir. Sistem mühendisliğinin temel amacı ise verimli, güvenilir ve maliyet etkin çözümler üretmek için disiplinler arası bir yaklaşımla işbirliğini sağlamak ve süreçleri optimize etmektir.",
    "V-Model, sistemin geliştirilme sürecinde kullanılan bir metodolojidir. Dokuz aşamadan oluşur ve süreç boyunca iteratif olarak uygulanabilir. Hataların erken tespiti ve maliyetleri düşürmek için sıkça tercih edilir.",
    "Sistemin gereksinimleri genellikle teknoloji, performans, güvenilirlik gibi teknik faktörleri kapsarken, kullanıcı gereksinimleri ise endüstriyel uygulamalar için gerekli işlevler, arayüz özellikleri veya kullanım kolaylığı gibi konuları içerir.",
    "İhtiyaç analizi, kullanıcı beklentilerini anlamak, proje ekibinin işbirliğini artırmak ve kaynakları etkin kullanmak için kritik bir süreçtir. İyi yapılan bir ihtiyaç analizi, sistemin başarılı olma olasılığını artırır.",
    "Trade-off analizi, farklı alternatifler arasından en iyi seçeneği belirlemek için kullanılan bir yöntemdir. Performans, maliyet, güvenlik gibi faktörleri dengeler ve proje başarısını artırır."
]

# Değerlendirme için veri setini oluştur
dataset = []
for question, deepseek_response, reference in zip(questions, deepseek_answers, reference_answers):
    dataset.append(
        {
            "user_input": question,
            "retrieved_contexts": [],  # Bağlam kullanmadığımız için boş liste
            "response": deepseek_response,
            "reference": reference
        }
    )

# Dataset'i Ragas ile formatla
evaluation_dataset = EvaluationDataset.from_list(dataset)

# Ragas değerlendirmesini gerçekleştir
result = evaluate(
    dataset=evaluation_dataset,
    metrics=[
        AnswerRelevancy(),  # Cevap soruya uygun mu?
        Faithfulness(),  # Cevap uydurma içeriyor mu?
        FactualCorrectness(),  # Cevap teknik olarak doğru mu?
        AnswerCorrectness()  # Cevap dilbilgisi ve mantık açısından doğru mu?
    ],
    llm=evaluator_llm
)

# Sonuçları yazdır
print("\n--- DeepSeek-r1 Değerlendirme Sonuçları (ChatGPT-4o Referans Olarak Kullanıldı) ---\n")
print(result)
